# **CREACIÓN DE LOS DATAFRAME**

### Importar las librerías

In [1]:
import pandas as pd
import numpy as np

### DataFrame Teams, Teams_History, Location y Arena

In [2]:
# Cargar los CSV a los dfs con las columnas que vamos a ocupar
df_team = pd.read_csv('csv/team.csv', usecols=['id', 'full_name','abbreviation','city', 'state'])
df_team_details = pd.read_csv('csv/team_details.csv', usecols=['team_id','owner', 'generalmanager','headcoach', 'arena', 'arenacapacity'])
df_team_history = pd.read_csv('csv/team_history.csv', usecols=['team_id', 'year_founded', 'year_active_till'])

new_teams_data = {
    'team_id': [1610612738, 1610612739, 1610612752, 1610612740, 1610612753],
    'owner': ['Wyc Grousbeck, Steve Pagliuca, y otros socios', 'Dan Gilbert', 'James Dolan (Madison Square Garden Sports)', 'Gayle Benson', 'RDV Sports, Inc. (familia DeVos)'],
    'generalmanager': ['Brad Stevens', 'Mike Gansey', 'Scott Perry', 'Trajan Langdon', 'Anthony Parker'],
    'headcoach': ['Joe Mazzulla', 'J.B. Bickerstaff', 'Tom Thibodeau', 'Willie Green', 'Jamahl Mosley'],
    'arena': ['TD Garden', 'Rocket Mortgage FieldHouse', 'Madison Square Garden', 'Smoothie King Center', 'Amway Center'],
    'arenacapacity': [19156, 19432, 19812, 16867, 18846]
}

# Crear DataFrame con nuevos datos
df_new_teams = pd.DataFrame(new_teams_data)

# Concatenar DataFrames
df_team_details = pd.concat([df_team_details, df_new_teams], ignore_index=True)

# Renombrar la columna id a team_id
df_team = df_team.rename(columns={'id': 'team_id'})

# Fusionar las tablas df_team y df_team_details
df_fusionado = pd.merge(df_team, df_team_details, on='team_id', how='outer')

# Fusionar la tabla resultante anterior con df_team_history
df_fusionado2 = pd.merge(df_fusionado, df_team_history, on='team_id', how='outer')

teams = df_fusionado2.rename(columns={'team_id': 'id'})

# Encontrar el valor máximo de la columna 'year_active_till'
max_year_active_till = teams['year_active_till'].max()

# Agregar la columna 'estado' basada en la condición
teams['estado'] = np.where(teams['year_active_till'] < max_year_active_till, 0, 1)

# Agregado de la tabla location al dataframe
# Crear un nuevo DataFrame con las columnas "state" y "city"
location = pd.DataFrame(teams, columns=['state','city'])

# Eliminar filas duplicadas basadas en la columna "city"
df_location_sin_duplicados = location.drop_duplicates(subset=['city'])

# Crear un nuevo DataFrame con las columnas
location = df_location_sin_duplicados

# Generar una columna "ID" con valores únicos
location['ID_location'] = range(1, len(location) + 1) 

# Unir los DataFrames en función de las columnas "city" y "state"
merge_location = pd.merge(teams, location, on=['city','state'], how='left')

teams = merge_location

teams = teams.drop(columns=['city','state'])

##Agregado de la tabla arena al dataframe
# Crear un nuevo DataFrame con las columnas "arena" y "arenacapacity"
arena = pd.DataFrame(teams, columns=['arena','arenacapacity'])

# Eliminar filas duplicadas basadas en la columna "city"
df_arena_sin_duplicados = arena.drop_duplicates(subset=['arena'])

# Crear un nuevo DataFrame con las columnas
arena = df_arena_sin_duplicados

# Generar una columna "ID" con valores únicos
arena['ID_arena'] = range(1, len(arena) + 1) 

# Unir los DataFrames en función de las columnas "city" y "state"
merge_arena = pd.merge(teams, arena, on=['arenacapacity','arena'], how='left')

teams = merge_arena

teams.rename(columns={"id": "ID_team"}, inplace=True)

teams = teams.drop(columns=['arenacapacity','arena','year_founded','year_active_till','estado'])

teams_History = df_team_history.rename(columns={'team_id': 'ID_team'})

# Generar una columna "ID" con valores únicos
teams_History['ID_history'] = range(1, len(teams_History) + 1)

teams = teams.drop_duplicates(subset='ID_team')

teams.fillna('Darko Rajaković', inplace=True)

### Dataframe de City y State

In [3]:
# Df city
city_unique = df_team['city'].unique()  # Obtiene los valores únicos de la columna 'city'
city = pd.DataFrame(city_unique, columns=['city'])  # Crea un DataFrame con los valores únicos

city['ID_city'] = range(1, len(city) + 1)  #Creo el indice

# Df state
state_unique = df_team['state'].unique()  # Obtiene los valores únicos de la columna 'state'
state = pd.DataFrame(state_unique, columns=['state'])  # Crea un DataFrame con los valores únicos

state['ID_state'] = range(1, len(state) + 1)  #Creo el indice

#Megere
# Unir los DataFrames en función de las columnas "city" y "state"
merge_city = pd.merge(city, location, on=['city'], how='left')
merge_total = pd.merge(state, merge_city, on=['state'], how='left')

merge_total = merge_total.drop(columns=['state','city'])

#Borro columnas de location
location = merge_total


### DataFrame Draft y Proveniencia

In [4]:
# Leer los archivos CSV
df_draft_combine_stats = pd.read_csv('csv/draft_combine_stats.csv')
df_draft_history = pd.read_csv('csv/draft_history.csv')

# Seleccion de las columnas a utilizar
columnas_draft_combine_stats  = ['player_id','weight','wingspan','standing_reach','body_fat_pct','standing_vertical_leap','max_vertical_leap','lane_agility_time','modified_lane_agility_time','three_quarter_sprint','bench_press']
df_draft_combine_stats_reducido = df_draft_combine_stats.loc[:,columnas_draft_combine_stats]

columnas_draft_history  = ['person_id','season','round_number','round_pick','overall_pick','team_id','organization','organization_type']
df_draft_history_reducido = df_draft_history.loc[:,columnas_draft_history]

df_draft_history_reducido.rename(columns={"person_id": "player_id"}, inplace=True)

# DataFrame Draft
draft = pd.merge(df_draft_combine_stats_reducido, df_draft_history_reducido, on='player_id', how='outer')

# Df proveniencia
# Crear un nuevo DataFrame con las columnas "organization" y "organization_type"
proveniencia = pd.DataFrame(draft, columns=['organization','organization_type'])  

# Eliminar filas duplicadas basadas en la columna "organization"
df_sin_duplicados = proveniencia.drop_duplicates(subset=["organization"])

# Crear un nuevo DataFrame con las columnas 
proveniencia = df_sin_duplicados

# Generar una columna "ID" con valores únicos
proveniencia["ID_proveniencia"] = range(1, len(proveniencia) + 1)

# Unir los DataFrames en función de las columnas "organization" y "organization_type"
merge = pd.merge(draft, proveniencia, on=["organization", "organization_type"], how="left")

draft = merge

draft = draft.drop(columns=['organization','organization_type'])

draft = draft.rename(columns={'player_id':'ID_player', 'team_id':'ID_team'})

### DataFrame Player

In [8]:
# Leer el archivo CSV
df_common_player_info = pd.read_csv('csv/common_player_info.csv')
df_inactive_players = pd.read_csv('csv/inactive_players.csv')
df_player = pd.read_csv('csv/player.csv')

#DF Players
player = df_common_player_info.drop(columns=['display_first_last','school','country','display_last_comma_first','display_fi_last','player_slug','last_affiliation','rosterstatus','games_played_current_season_flag','team_name','team_abbreviation','team_code','team_city','playercode','from_year','to_year','dleague_flag','nba_flag','games_played_flag','draft_year','draft_round','draft_number','greatest_75_flag'])
# Renombrar la columna id a person_id
df_player = df_player.rename(columns={'id': 'person_id'})

# Fusionar las tablas df_player y player
player = pd.merge(player, df_player, on='person_id', how='inner')
player = player.drop(columns=['last_name_y','first_name_y','full_name'])

# Renombrar la columna id a ID_player
player = player.rename(columns={'person_id': 'ID_player', 'first_name_x':'first_name', 'last_name_x':'last_name', 'team_id':'ID_team'})

player = player[player['ID_team']>=1610612737]


### DataFrame Games, Game Stats y Season

In [6]:
# Cargar los archivos CSV
df_game = pd.read_csv('csv/game.csv')  
df_game_info = pd.read_csv('csv/game_info.csv')  
df_game_summary = pd.read_csv('csv/game_summary.csv')  
df_line_score = pd.read_csv('csv/line_score.csv')

# Unir las tablas
df_combine = df_game.merge(df_game_info, on='game_id', how='inner')

df_combine_summary = df_combine.merge(df_game_summary, on='game_id', how='inner')

df_combine_total = df_combine_summary.merge(df_line_score, on='game_id', how='inner')

# Seleccionar las columnas

game_stats = df_combine_total[['game_id','pts_home_x', 'pts_qtr1_home', 'pts_qtr2_home', 'pts_qtr3_home', 'pts_qtr4_home', 
                                  'pts_qtr1_away', 'pts_qtr2_away', 'pts_qtr3_away', 'pts_qtr4_away', 'pts_away_x', 
                                 'attendance', 'game_time', 'natl_tv_broadcaster_abbreviation', 'live_period_time_bcast']]

game_stats['ID_game_stats'] = range(1, len(game_stats) + 1)  # Creación del id


df_games = df_combine_total[['game_id', 'game_date_x', 'team_id_home_x', 'wl_home', 'team_id_away_x', 'wl_away', 'season_id', 'season', 'season_type']]

# Renombrar la columna team_id_home a ID_team en df_games
df_games = df_games.rename(columns={'team_id_home_x': 'ID_team'})

games = pd.merge(df_games, teams[['ID_team', 'ID_location']], on='ID_team', how='outer')


# Renombrar la columna game_id a ID_game
games = games.rename(columns={'game_id': 'ID_game', 'game_date_x': 'game_date', 'team_id_away_x': 'team_id_away'})


# Creación de tabla de Season 
df_season = df_combine_total[['season_id', 'season', 'season_type']]
season = df_season.drop_duplicates(subset=['season_id'])
season = season.rename(columns={'season_id': 'ID_season'})
season = season.drop_duplicates(subset='ID_season')

games = games.rename(columns={'season_id': 'ID_season'})
games = games.drop(columns=['season','season_type'])
games = games.drop_duplicates(subset='ID_game')

game_stats = game_stats.rename(columns={'game_id': 'ID_game'})

# ID_season en tabla de Draft
draft = pd.merge(draft, season[['ID_season', 'season']], on='season', how='inner')
draft_d = draft.drop(columns=['season'])
draft = draft_d

draft['ID_draft'] = range(1, len(draft) + 1)  # Creación del id

C:\Users\Usuario\AppData\Local\Temp\ipykernel_24908\2576522411.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_stats['ID_game_stats'] = range(1, len(game_stats) + 1)  # Creación del id


### Carga de los datos a SQL Server

In [7]:
from sqlalchemy import create_engine

# Parámetros de la conexión
server = 'localhost\\SQLEXPRESS'  # Reemplaza con el nombre de tu servidor
database = 'DAFT01_Grupo1_nba'  # Reemplaza con el nombre de tu base de datos

# Crear la cadena de conexión utilizando la autenticación de Windows
connection_string = f'mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes'

# Crear el motor de conexión
engine = create_engine(connection_string)

# Mandar los datos a la tabla ya creada en SQL Server
season.to_sql('season', engine, if_exists='append', index=False)
state.to_sql('State', engine, if_exists='append', index=False)
city.to_sql('City', engine, if_exists='append', index=False)
arena.to_sql('Arena', engine, if_exists='append', index=False)
location.to_sql('Location', engine, if_exists='append', index=False)
proveniencia.to_sql('Proveniencia', engine, if_exists='append', index=False)
teams.to_sql('teams', engine, if_exists='append', index=False)
teams_History.to_sql('teams_History', engine, if_exists='append', index=False)
games.to_sql('Games', engine, if_exists='append', index=False)

# Estos dan error porque falta corregir el tipo de dato que hay en las tablas o en el dataframe
game_stats.to_sql('Game_stats', engine, if_exists='append', index=False)
draft.to_sql('Draft', engine, if_exists='append', index=False)
player.to_sql('Player', engine, if_exists='append', index=False)

IntegrityError: (pyodbc.IntegrityError) ('23000', "[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Violation of PRIMARY KEY constraint 'PK__season__70D78338A91960E8'. Cannot insert duplicate key in object 'dbo.season'. The duplicate key value is (21946). (2627) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)")
[SQL: INSERT INTO season ([ID_season], season, season_type) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (? ... 2184 characters truncated ... ), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?), (?, ?, ?)]
[parameters: (21946, 1946, 'Regular Season', 41946, 1946, 'Playoffs', 21947, 1947, 'Regular Season', 41947, 1947, 'Playoffs', 21948, 1948, 'Regular Season', 41948, 1948, 'Playoffs', 21949, 1949, 'Regular Sea